# Machine Learning Fairness Algorithms Evaluation

#### Fairness in Machine Learning
The aim of a fairness algorithm is to avoid the outcome decisions are being made unfairly to certain groups or individuals.
- Algorithms are made by humans and trained by data which may be biased
- However, there are many definitions of fairness that cannot be optimized at the same time


#### Learning fair representations (LFR)
The main idea: map each individual, represented as a data point in a given input space, to a probability distribution in a new representation space. The aim of this new representation is to lose any information that can identify whether the person belongs to the protected subgroup, while retaining as much other information as possible.

A discriminative clustering model, where the prototypes act as the clusters

#### Prejudice Remover Regularizer (PR)

ADD DETAILS


### Data Pre-processing

In [1]:
import csv
import pandas as pd

In [7]:
data = 'https://raw.githubusercontent.com/juliamblake1/ADS-Project-4/main/data/compas-scores.csv'

In [8]:
compas_scores = pd.read_csv(data,header =0)
compas_scores.head()

FileNotFoundError: [Errno 2] No such file or directory: '.../data/compas-scores.csv'

In [4]:
compas_scores_race = compas_scores.loc[compas_scores['race'].isin(['Caucasian' ,'African-American'])]
compas_scores_race

KeyError: 'race'

In [ ]:
# converting to binary data
df_one = pd.get_dummies(compas_scores_race["race"])
# print(df_one)
 
# display result
df_two = pd.concat((df_one, compas_scores_race), axis=1)
df_two = df_two.drop(["race"], axis=1)
df_two = df_two.drop(["Caucasian"], axis=1)
new_df = df_two.rename(columns={"African-American": "race"})
new_df